##### Let’s try to redo last exercise, but this time let’s use make_pipeline() and make_column_transformer.

#### Tasks:

##### Import the necessary library.
##### For all pipelines, make sure to use make_pipeline() where possible.
##### Create a pipeline for the numeric features. It should have the first step as simple imputation using strategy="median" and the second step should be using StandardScaler. Name this pipeline numeric_transformer.
##### Create a pipeline for the categorical features. It should also have 2 steps. The first is imputation using strategy="most_frequent". The second step should be one-hot encoding with handle_unknown="ignore". Name this pipeline categotical_transformer.
##### Make your column transformer named col_transformer by using make_column_transformer()and specify the transformations on numeric_features and categorical_features using the appropriate pipelines you build above.
##### Create a main pipeline named main_pipe which preprocesses with col_transformer followed by building a KNeighborsRegressor model.
##### The last step is performing cross-validation using our pipeline.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.neighbors import KNeighborsRegressor

##### Loading in the data

In [2]:
bball_df = pd.read_csv('bball_imp.csv').dropna(subset=['salary'])

##### Split the dataset

In [3]:
df_train, df_test = train_test_split(bball_df, test_size=0.2, random_state=7)

X_train = df_train[["weight", "height", "draft_year", "draft_round",
                     "draft_peak", "team", "position", "country"]]
X_test = df_test[["weight", "height", "draft_year", "draft_round",
                     "draft_peak", "team", "position", "country"]]
y_train = df_train['salary']
y_test = df_test['salary']

##### Split the numeric and categorical features 

In [4]:
numeric_features = [ "weight",
                     "height",
                     "draft_year",
                     "draft_round",
                     "draft_peak"]

categorical_features = ["team", "position", "country"]

##### Build a numeric pipeline

Create a pipeline for the numeric features. It should have the first step as simple imputation using strategy="median" and the second step should be using StandardScaler. Name this pipeline numeric_transformer.

In [5]:
numeric_transformer = make_pipeline(SimpleImputer(strategy='median'), StandardScaler())

##### Build a categorical pipeline

Create a pipeline for the categorical features. It should also have 2 steps. The first is imputation using strategy="most_frequent". The second step should be one-hot encoding with handle_unknown="ignore". Name this pipeline categotical_transformer.

In [6]:
categorical_transformer = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder(handle_unknown='ignore'))

##### Build a column transformer

Make your column transformer named col_transformer by using make_column_transformer()and specify the transformations on numeric_features and categorical_features using the appropriate pipelines you build above.

In [8]:
col_transformer = make_column_transformer(
    (numeric_transformer, numeric_features),
    (categorical_transformer, categorical_features)
)

##### Build a main pipeline

Create a main pipeline named main_pipe which preprocesses with col_transformer followed by building a KNeighborsRegressor model.

In [9]:
main_pipe = make_pipeline(col_transformer, KNeighborsRegressor())

##### Cross validate

The last step is performing cross-validation using our pipeline.

In [10]:
with_categorical_scores = cross_validate(main_pipe, X_train, y_train, return_train_score=True)
pd.DataFrame(with_categorical_scores)

,fit_time,score_time,test_score,train_score
0,0.024040,0.013961,0.338072,0.599277
1,0.007896,0.005420,0.207216,0.646728
2,0.006895,0.004696,0.492646,0.576790
3,0.006430,0.005055,0.483127,0.596786
4,0.009768,0.005053,0.476496,0.581861
